In [ ]:
from utils import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report as clsr

In [ ]:
from abstractcls import ABSClassifier

In [ ]:
pubs_dict, all_abs = get_pubs_dict()

In [ ]:
cls = ABSClassifier()

In [ ]:
# dict to dataframe
pubs_df = pd.DataFrame(pubs_dict).transpose()

In [ ]:
# count journals and leave only those having more than 200 publications
counts = pubs_df.groupby('journal').count()
journals = counts[counts['abstract'] > 200].index.values

In [ ]:
#sns bar plot number of abstract per journal 

df =counts[counts['abstract'] > 200].sort_values(by='abstract',ascending=False).reset_index()
sns.set(style="whitegrid")
f, ax = plt.subplots(figsize=(7,7))
sns.set_color_codes("pastel")
sns.barplot(x="abstract", y="journal", data=df, color="b",)
ax.set(xlim=(0, 3500), ylabel="",xlabel="Total number of abstracts")
sns.despine(left=True, bottom=True)
#ax.set_yticklabels(labels = ax.get_yticklabels(),   fontdict =  {
#            'horizontalalignment': 'left'})
#ax.axes.yaxis.set_zorder(-10.0)
#ax.margin(100.0)
ax.set_ylabel("Journal")
plt.savefig('number_by_journal.pdf',bbox_inches='tight',pad_inches=1.0)
plt.show()

In [ ]:
# select publication with valid abstracts
pubs_select = pubs_df[np.isin(pubs_df['journal'],journals)]
# get train date and labels
X,y = pubs_select["abstract"].values,pubs_select["journal"].values

In [ ]:
cls.fit(X,y)

In [ ]:
val_dict, _ = get_pubs_dict(fnames = ['data/abstracts_2018_01.txt','data/abstracts_2018_02.txt','data/abstracts_2018_03.txt'])
val_df = pd.DataFrame(val_dict).transpose()

In [ ]:
val_select = val_df[(val_df['abstract'] != ('Not Available')) & (np.isin(val_df['journal'],journals))]

In [ ]:
Xval = val_select['abstract'].values
yval = val_select['journal'].values
yval_pred = cls.model.predict(Xval)

In [ ]:
classes_dict  = {}
for i,cl in enumerate(cls.labels_.classes_): 
    classes_dict[cl] = i

In [ ]:
yval_labels = [classes_dict[y] for y in yval] 

In [ ]:
print(clsr(yval_labels, yval_pred, target_names=cls.labels_.classes_))